In [1]:
# third-party
import pandas as pd
import numpy as np
import scipy
import pickle

# local
from get_data import get_performance_metrics, get_delays, transform_overview_on_target, transform_overview_on_overall
from fr_performance import get_fr_detection_performance
from delay_performance import normality_test, plot_inspiration_vs_expiration


### Activity-specific only FR analysis

In [2]:
# Transform overview from participant specific into all participants
with open('Results/results.pickle', 'rb') as file:
    overview = pickle.load(file)

overview_all_participants = transform_overview_on_target(overview, target="Activity")
overview_all_participants.keys()

dict_keys(['SNB', 'SGB', 'MIXB', 'STNB', 'MCH', 'SQT', 'AAL', 'AAR', 'ALL', 'ALR', 'UAL', 'UAR', 'SE', 'SS', 'TR', 'SNBm', 'UALm', 'UARm'])

In [3]:
performance_all_participants = get_fr_detection_performance(overview_all_participants, target="Activity")
performance_all_participants

,Activity,Sensor,Ratio,Precision,Recall,Mean absolute delay $\pm$ SD,Adjusted delay
0,SNB,MAG,1.02,0.97,0.99,-0.07 $\pm$ 0.43,-1 %
1,SNB,PZT,1.14,0.87,0.98,-0.39 $\pm$ 0.58,-9 %
2,SGB,MAG,1.00,0.94,0.94,-0.23 $\pm$ 0.95,-2 %
3,SGB,PZT,1.62,0.60,0.98,-0.87 $\pm$ 1.68,-8 %
4,MIXB,MAG,0.99,0.99,0.98,-0.07 $\pm$ 0.4,-1 %
5,MIXB,PZT,1.12,0.84,0.94,-0.47 $\pm$ 0.86,-9 %
6,STNB,MAG,1.05,0.95,0.99,-0.08 $\pm$ 0.37,-1 %
7,STNB,PZT,1.05,0.93,0.98,-0.31 $\pm$ 0.86,-6 %
8,MCH,MAG,1.17,0.85,0.99,-0.11 $\pm$ 0.4,-3 %
9,MCH,PZT,1.59,0.61,0.96,-0.09 $\pm$ 0.69,-2 %


In [4]:
mag = performance_all_participants[performance_all_participants["Sensor"] == "MAG"].loc[:,["Ratio", "Precision", "Recall"]].reset_index(drop=True) 
pzt = performance_all_participants[performance_all_participants["Sensor"] == "PZT"].loc[:,["Ratio", "Precision", "Recall"]].reset_index(drop=True)

a = mag / pzt 
a["Activity"] = performance_all_participants[performance_all_participants["Sensor"] == "MAG"].reset_index(drop=True)["Activity"]
a

,Ratio,Precision,Recall,Activity
0,0.894737,1.114943,1.010204,SNB
1,0.617284,1.566667,0.959184,SGB
2,0.883929,1.178571,1.042553,MIXB
3,1.000000,1.021505,1.010204,STNB
4,0.735849,1.393443,1.031250,MCH
5,0.770833,1.308824,1.010204,SQT
6,0.856000,1.151899,0.989899,AAL
7,0.808000,1.240506,1.000000,AAR
8,0.764706,1.338028,1.010309,ALL
9,0.801587,1.376812,1.103448,ALR


In [5]:
mag["Activity"] = performance_all_participants[performance_all_participants["Sensor"] == "MAG"].reset_index(drop=True)["Activity"]
mag

,Ratio,Precision,Recall,Activity
0,1.02,0.97,0.99,SNB
1,1.00,0.94,0.94,SGB
2,0.99,0.99,0.98,MIXB
3,1.05,0.95,0.99,STNB
4,1.17,0.85,0.99,MCH
5,1.11,0.89,0.99,SQT
6,1.07,0.91,0.98,AAL
7,1.01,0.98,0.99,AAR
8,1.04,0.95,0.98,ALL
9,1.01,0.95,0.96,ALR


### Participant-specific only FR analysis

In [6]:
# Transform overview from participant specific into all participants
with open('Results/results.pickle', 'rb') as file:
    overview = pickle.load(file)

overview_all_activities = transform_overview_on_target(overview, target="ID")
overview_all_activities.keys()

dict_keys(['7OYX', 'NO15', 'G8B7', 'EPE2', 'HAK8', '1BST', '83J1', 'QMQ7', '9TUL', 'FTD7', 'Y6O3', '2QWT', 'F9AF', 'P4W9', 'W8Z9', 'D4GQ'])

In [7]:
get_fr_detection_performance(overview_all_activities, target="ID")

,ID,Sensor,Ratio,Precision,Recall,Mean absolute delay $\pm$ SD,Adjusted delay
0,7OYX,MAG,1.07,0.93,1.00,-0.06 $\pm$ 0.38,-1 %
1,7OYX,PZT,1.82,0.53,0.97,-0.3 $\pm$ 0.91,-6 %
2,NO15,MAG,1.17,0.84,0.99,-0.05 $\pm$ 0.43,-1 %
3,NO15,PZT,1.48,0.66,0.97,-0.38 $\pm$ 0.77,-9 %
4,G8B7,MAG,1.12,0.85,0.95,-0.13 $\pm$ 0.56,-4 %
5,G8B7,PZT,1.05,0.94,0.98,-0.44 $\pm$ 0.45,-13 %
6,EPE2,MAG,1.03,0.94,0.97,0.18 $\pm$ 0.49,4 %
7,EPE2,PZT,1.25,0.75,0.95,-0.46 $\pm$ 0.81,-10 %
8,HAK8,MAG,1.09,0.90,0.99,-0.09 $\pm$ 0.34,-2 %
9,HAK8,PZT,1.42,0.66,0.93,0.09 $\pm$ 0.75,2 %


### Overall FR Analysis

In [8]:
# Transform overview from participant specific into all participants
with open('Results/results.pickle', 'rb') as file:
    overview = pickle.load(file)

overview_all = transform_overview_on_overall(overview)
overview_all.keys()

dict_keys(['ScientISST', 'BIOPAC', 'BITalino'])

In [9]:
get_fr_detection_performance(overview_all, target=None)

,Sensor,Ratio,Precision,Recall,Mean absolute delay $\pm$ SD,Adjusted delay
0,MAG,1.11,0.88,0.98,-0.04 $\pm$ 0.46,-1 %
1,PZT,1.33,0.72,0.95,-0.22 $\pm$ 0.79,-4 %


### Overall delay analysis

In [10]:
delays_i_all_scientisst, delays_e_all_scientisst = [], []
delays_i_all_bitalino, delays_e_all_bitalino = [], []

for activity in overview_all_participants.keys():

    delays_i_all_scientisst += overview_all_participants[activity]["ScientISST"]["delay_i"] 
    delays_e_all_scientisst = overview_all_participants[activity]["ScientISST"]["delay_e"]

    delays_i_all_bitalino += overview_all_participants[activity]["BITalino"]["delay_i"]
    delays_e_all_bitalino = overview_all_participants[activity]["BITalino"]["delay_e"]

In [11]:
print(f"min delay scientisst {min(delays_i_all_scientisst + delays_e_all_scientisst, key=abs)}")
print(f"min delay bitalino {min(delays_i_all_bitalino + delays_e_all_bitalino, key=abs)}")

print(f"max delay scientisst {max(delays_i_all_scientisst + delays_e_all_scientisst, key=abs)}")
print(f"max delay bitalino {max(delays_i_all_bitalino + delays_e_all_bitalino, key=abs)}")

print(f"mean delay scientisst {round(np.mean(delays_i_all_scientisst + delays_e_all_scientisst), 2)} +/- {np.round(np.std(delays_i_all_scientisst + delays_e_all_scientisst), 2)}")
print(f"mean delay bitalino {np.round(np.mean(delays_i_all_bitalino + delays_e_all_bitalino), 2)} +/- {np.round(np.std(delays_i_all_bitalino + delays_e_all_bitalino), 2)}")

min delay scientisst -0.0
min delay bitalino -0.0
max delay scientisst -3.22
max delay bitalino -4.92
mean delay scientisst -0.05 +/- 0.52
mean delay bitalino -0.24 +/- 0.82


In [12]:
normality_test(delays_i_all_scientisst + delays_e_all_scientisst, sensor="MAG", type="inspiration+expiration")
normality_test(delays_i_all_bitalino + delays_e_all_bitalino, sensor="PZT", type="inspiration+expiration")

### Event-specific delay analysis

In [13]:
delays_df = pd.DataFrame(columns=["Sensor", "Type", "Delay"])
delays_df["Delay"] = delays_i_all_scientisst + delays_e_all_scientisst + delays_i_all_bitalino + delays_e_all_bitalino
delays_df["Sensor"] = ["MAG"] * len(delays_i_all_scientisst + delays_e_all_scientisst) + ["PZT"] * len(delays_i_all_bitalino + delays_e_all_bitalino)
delays_df["Type"] = ["Inhalation"] * len(delays_i_all_scientisst) + ["Exhalation"] * len(delays_e_all_scientisst) + ["Inhalation"] * len(delays_i_all_bitalino) + ["Exhalation"] * len(delays_e_all_bitalino)
delays_df


,Sensor,Type,Delay
0,MAG,Inhalation,-0.22
1,MAG,Inhalation,-0.05
2,MAG,Inhalation,-0.11
3,MAG,Inhalation,0.01
4,MAG,Inhalation,0.12
...,...,...,...
9146,PZT,Exhalation,-0.38
9147,PZT,Exhalation,-0.11
9148,PZT,Exhalation,-0.38
9149,PZT,Exhalation,-0.33


In [14]:
plot_inspiration_vs_expiration(delays_df)